In [ ]:
%pip install rustxes

In [ ]:
%pip install "pm4py==2.7.8.4"

In [4]:
rtfm_xes_path = "/home/aarkue/doc/sciebo/alpha-revisit/Road_Traffic_Fine_Management_Process.xes.gz"

In [6]:
import pm4py
log1 = pm4py.read_xes(rtfm_xes_path)
log1.head(5)

parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount,matricola
0,35.0,561,NIL,Create Fine,A,0.0,complete,2006-07-23 22:00:00+00:00,157.0,0.0,A1,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-04 23:00:00+00:00,NaN,NaN,A1,11.0,NaN,NaN,NaN,NaN
2,35.0,561,NIL,Create Fine,A,0.0,complete,2006-08-01 22:00:00+00:00,157.0,0.0,A100,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Send Fine,NaN,NaN,complete,2006-12-11 23:00:00+00:00,NaN,NaN,A100,11.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Insert Fine Notification,NaN,NaN,complete,2007-01-14 23:00:00+00:00,NaN,NaN,A100,NaN,P,P,NaN,NaN


In [8]:
import pm4py
log2 = pm4py.read_xes(rtfm_xes_path,variant="rustxes")
log2[log1.columns].head(5)

,amount,org:resource,dismissal,concept:name,vehicleClass,totalPaymentAmount,lifecycle:transition,time:timestamp,article,points,case:concept:name,expense,notificationType,lastSent,paymentAmount,matricola
0,35.0,561,NIL,Create Fine,A,0.0,complete,2006-07-23 22:00:00+00:00,157.0,0.0,A1,NaN,None,None,NaN,NaN
1,NaN,None,None,Send Fine,None,NaN,complete,2006-12-04 23:00:00+00:00,NaN,NaN,A1,11.0,None,None,NaN,NaN
2,35.0,561,NIL,Create Fine,A,0.0,complete,2006-08-01 22:00:00+00:00,157.0,0.0,A100,NaN,None,None,NaN,NaN
3,NaN,None,None,Send Fine,None,NaN,complete,2006-12-11 23:00:00+00:00,NaN,NaN,A100,11.0,None,None,NaN,NaN
4,NaN,None,None,Insert Fine Notification,None,NaN,complete,2007-01-14 23:00:00+00:00,NaN,NaN,A100,NaN,P,P,NaN,NaN


In [ ]:
import rust_bridge_pm_py

In [ ]:
import pm4py
import polars
import rust_bridge_pm_py
xes_path = "/home/aarkue/doc/sciebo/alpha-revisit/Sepsis Cases - Event Log.xes"

In [ ]:
# Generate huge event log (on Python side, but structs reside in Rust!)
log = rust_bridge_pm_py.native.PyBridgeEventLog()
for i in range(200000):
  trace = rust_bridge_pm_py.native.PyBridgeTrace("Trace " + str(i))
  for j in range(15):
    event = rust_bridge_pm_py.native.PyBridgeEvent({"concept:name": "Activity " + str(j)})
    trace.insert_event(j,event)
  log.insert_trace(i,trace)

In [ ]:
all_events = 0
all_events_with_same_timestamp_in_case = 0 
for g,r in log_df.groupby(["case:concept:name"]):
  events_with_same_timestamp_in_case = r.shape[0] - len(r['time:timestamp'].unique())
  all_events += r.shape[0]
  all_events_with_same_timestamp_in_case += events_with_same_timestamp_in_case
  assert events_with_same_timestamp_in_case >= 0 
  # print(events_with_same_timestamp_in_case)
print(all_events_with_same_timestamp_in_case/all_events)

In [ ]:
# Read some event log
log_df = pm4py.read_xes(xes_path,variant="line_by_line")

In [ ]:
res_log = rust_bridge_pm_py.import_xes(xes_path)[0].to_pandas()

In [ ]:
import polars as pl
res_log.select([pl.col("time:timestamp").dt.replace_time_zone("UTC")])

In [ ]:

FIGURE_PATH = "./figures/"
EVENT_LOGS_PATH = "/home/aarkue/doc/sciebo/alpha-revisit/"
log_to_path = {
    "BPI_Challenge_2020_DomesticDeclarations": f"{EVENT_LOGS_PATH}DomesticDeclarations.xes.gz",
    "BPI_Challenge_2020": f"{EVENT_LOGS_PATH}BPI_Challenge_2020_request_for_payments.xes",
    "BPI_Challenge_2019_sampled_3000cases": f"{EVENT_LOGS_PATH}BPI_Challenge_2019_sampled3000.xes",
    "Sepsis": f"{EVENT_LOGS_PATH}Sepsis Cases - Event Log.xes.gz",
    "RTFM": f"{EVENT_LOGS_PATH}Road_Traffic_Fine_Management_Process.xes.gz",
}



In [ ]:
from pandas import DataFrame
from numpy import isnan
def compare_log_dfs(df1: DataFrame,df2: DataFrame):
  all_columns = set(df1.columns).union(set(df2.columns))
  all_equal = True
  for col in all_columns:
    # or all(...) to handle some weird datatype stuff; if all values are equal, we also consider the column as equal
    if  df1[col].equals(df2[col]) or all([v1 == v2 or (isnan(v1) and isnan(v2)) for (v1,v2) in zip(df1[col],df2[col])]):
      # print(f"✅ For column {col} all entries are equal!")
      pass
    else:
      all_equal = False
      print(f"❌ For column {col} not all entries are equal!")
      print(df1[col],df2[col])
  if all_equal:
    print(f"✅✅✅ All values in all column are equal!")
    return True
  else:
    print(f"❌❌❌ NOT all values in all column are equal!")
    return False


In [ ]:
import time
import pm4py
import numpy as np
log_import_res = dict()
for log_name in log_to_path:
  print(log_name)
  start = time.time()
  log_rs = rust_bridge_pm_py.native.import_xes(log_to_path[log_name]).to_pandas()
  log_rs = log_rs.fillna(np.nan)
  rs_dur = time.time() - start
  start = time.time()
  log_py_lbl = pm4py.read_xes(log_to_path[log_name],variant="line_by_line")
  py_lbl_dur = time.time() - start
  start = time.time()
  log_py_iter = pm4py.read_xes(log_to_path[log_name],variant="iterparse")
  py_iter_dur = time.time() - start
  print(rs_dur,py_lbl_dur,py_iter_dur)
  log_import_res[log_name] = {"Rust": rs_dur, "PM4Py (line_by_line)": py_lbl_dur, "PM4Py (iterparse)": py_iter_dur }
  if not compare_log_dfs(log_rs,log_py_lbl):
    break


In [ ]:
import json
json.dumps(log_import_res)

In [ ]:
from typing import Tuple
from pm4py.objects.petri_net.obj import PetriNet, Marking

# Checks if two Accepting Petri Nets _could_ be equal (syntactically)
# Best-effort check, does not compute full isomorphism between nets
# Partly ignores silent transitions, so use with caution
def pns_could_be_equal(a: Tuple[PetriNet,Marking,Marking], b: Tuple[PetriNet,Marking,Marking]):
  a = list(a)
  b = list(b)
  failures_reasons = []
  if len(a[0].transitions) != len(b[0].transitions):
    failures_reasons.append(f"Different number of transitions: {len(a[0].transitions)} != {len(b[0].transitions) }")
  if len(a[0].places) != len(b[0].places):
    failures_reasons.append(f"Different number of places: {len(a[0].places)} != {len(b[0].places) }")
  if len(a[0].arcs) != len(b[0].arcs):
    failures_reasons.append(f"Different number of arcs: {len(a[0].arcs)} != {len(b[0].arcs) }")
  transition_labels_a = [str(t.label) for t in a[0].transitions]
  transition_labels_b = [str(t.label) for t in b[0].transitions]
  if sorted(transition_labels_a) != sorted(transition_labels_b):
    failures_reasons.append(f"Transitions labels are not equal {list(sorted(transition_labels_a))} != {list(sorted(transition_labels_a))}")
  def extract_place_candidates(pn: PetriNet):
    candidates = {p: (set(),set()) for p in pn.places}
    for arc in pn.arcs:
      if type(arc.source) == PetriNet.Place:
        candidates[arc.source][0].add(str(arc.target.label))
      elif type(arc.source) == PetriNet.Transition:
        candidates[arc.target][1].add(str(arc.source.label))
    return {(frozenset(in_trans),frozenset(out_trans)) for (in_trans,out_trans) in candidates.values()}
  place_candidates_a = extract_place_candidates(a[0])
  place_candidates_b = extract_place_candidates(b[0])
  if place_candidates_a != place_candidates_b:
    failures_reasons.append(f"Different place candidates: {place_candidates_a} != {place_candidates_b}")
  if a[1] is None:
    a[1] = Marking()
  if a[2] is None:
    a[2] = Marking()
  if b[1] is None:
    b[1] = Marking()
  if b[2] is None:
    b[2] = Marking()
  if sorted(list(a[1].values())) != sorted(list(b[1].values())):
    failures_reasons.append(f"Initial Markings do not match: {a[1]} != {b[1]}")
  if sorted(list(a[2].values())) != sorted(list(b[2].values())):
    failures_reasons.append(f"Final Markings do not match: {a[2]} != {b[2]}")
  for reason in failures_reasons:
    print(reason)
  return len(failures_reasons) == 0


In [ ]:
import glob
ALPHAPPP_PNML_FOLDER = "/home/aarkue/doc/sciebo/alpha-revisit/report/report_figures/"
pn_path_per_algolog = dict()
right = 0
wrong = 0
for pnml_path in glob.glob(ALPHAPPP_PNML_FOLDER + "*.pnml"):
    pnml_name = pnml_path.split("/")[-1]
    split_name = pnml_name.split("_")
    split_name.pop() # Pop timestamp
    algo_name = split_name.pop() # Pop algo name
    split_name.pop() # Pop _model
    log_name = "_".join(split_name)
    print(log_name)
    print(pnml_name)
    pn_path_per_algolog[(log_name,algo_name)] = pnml_path
    log_path = log_to_path[log_name]
    pn = pm4py.read_pnml(pnml_path)
    pm4py.view_petri_net(*pn)
    
    # pm4py.view_petri_net(*pn)
    algo_config = algo_name[len("α+++")+1:].split("|")
    repair_thresh = float(algo_config[0])
    b = float(algo_config[1][1:])
    t = float(algo_config[2][1:])
    r = float(algo_config[3][1:])
    print({"b": b,"t":t,"r": r,})
    import json
    from rust_bridge_pm_py import petri_net
    config = {
    "balance_thresh": b,
    "fitness_thresh": t,
    "replay_thresh": r,
    "log_repair_skip_df_thresh_rel": repair_thresh,
    "log_repair_loop_df_thresh_rel": repair_thresh,
    "absolute_df_clean_thresh": 1,
    "relative_df_clean_thresh": 0.01,
    }
    (rust_net_json,algo_performance) = rust_bridge_pm_py.native.discover_net_alphappp(log_to_path[log_name],json.dumps(config))
    rust_net = petri_net.dict_to_petrinet(json.loads(rust_net_json))
    pm4py.view_petri_net(*rust_net)
    if pns_could_be_equal(pn,rust_net):
        print("=== EQUAL ====")
        right += 1
    else:
        print("===NOT EQUAL====")
        wrong += 1

In [ ]:
py_net_path = "/home/aarkue/doc/sciebo/alpha-revisit/final_figures/Sepsis_model_α+++ 4.0 b0.3 t0.7 r0.6_2023-04-02T22 22 36.885210.pnml"
py_net = pm4py.read_pnml(py_net_path)
pm4py.view_petri_net(*py_net)

In [ ]:
rust_net_path = "/home/aarkue/doc/projects/rust-bridge-process-mining/res-Sepsis Cases - Event Log.xes-α+++|4.0|b0.3|t0.7|.pnml"
rust_net = pm4py.read_pnml(rust_net_path)
pm4py.view_petri_net(*rust_net)

In [ ]:
# Sepsis_model_α+++|2.0|b0.3|t0.7|r0.6_2023-04-02T22:58:35.500530.pnml
import json
from rust_bridge_pm_py import petri_net
# xes_path = log_to_path["BPI_Challenge_2019_sampled_3000cases"]
config = {
  "balance_thresh": 0.5,
  "fitness_thresh": 0.5,
  "replay_thresh": 0.5,
  "log_repair_skip_df_thresh_rel": 4.0,
  "log_repair_loop_df_thresh_rel": 4.0,
  "absolute_df_clean_thresh": 1,
  "relative_df_clean_thresh": 0.01,
}
(rust_net_json,algo_performance) = rust_bridge_pm_py.native.discover_net_alphappp(xes_path,json.dumps(config))
rust_net = petri_net.dict_to_petrinet(json.loads(rust_net_json))
pm4py.view_petri_net(*rust_net)

In [ ]:
json.loads(algo_performance)

In [ ]:
acts = ["Admission IC", "Release B", "Release D", "Leucocytes", "LacticAcid", "Admission NC", "IV Antibiotics", "ER Triage", "ER Sepsis Triage", "CRP", "Release E", "IV Liquid", "Return ER", "ER Registration", "Release C", "Release A", "__START", "__END"]

print(log_df['case:concept:name'].unique().shape[0])
log_df[log_df["concept:name"] == acts[1]].shape[0]

In [ ]:
pm4py.precision_alignments(log_df,*rust_net)

In [ ]:
import json
from rust_bridge_pm_py import petri_net
trans_net = petri_net.petrinet_to_dict(net,im,[fm])
res_net_json = rust_bridge_pm_py.native.test_petrinet(json.dumps(trans_net))
res_net =  petri_net.dict_to_petrinet(json.loads(res_net_json))
pm4py.view_petri_net(*res_net)

In [ ]:
pns_could_be_equal(pn,rust_net)

In [ ]:
# First convert DF log to PyBridgeEventLog Wrapper and then add start/end acts.
# Result is again a PyBridgeEventLog Wrapper 
# Performance: Okay for smallish/normal logs but poor for very large ones
res_log = rust_bridge_pm_py.native.test_bridge_log(pm4py_log_to_bridge_log(log_df))

In [ ]:
# Idea: Do not require Polar dependency on python side, by exporting JSON with pandas
# and then importing it using Polar on the Rust side
log = rust_bridge_pm_py.native.test_df_pandas(log_df.to_json(orient="records"),"json")

In [ ]:
# Same as directly above, but using CSV:
log = rust_bridge_pm_py.native.test_df_pandas(log_df.to_csv(),"csv")

In [ ]:
# Idea: Do DataFrame -> Wrapper conversion on Rust side (+ in parallel)
# For that, polars is used (because of the first-class Rust support)
# First, convert log (pandas) DF to polars DF and then convert it to PyBridgeEventLog Wrapper in Rust, return Result 
# Performance: Pretty good :) 
log = rust_bridge_pm_py.native.polars_df_to_log(polars.from_pandas(log_df))

In [ ]:
# Create huge event log & process it in Rust
# For conversion, use json bytes (using orjson library)
# Not great performance...
# Total: 15308.481216430664ms; Json Dump & Log Re-construction from Dict takes the most time
# This prompted the experimentation with PyBridgeEventLog Wrapper, living in Rust
l = rust_bridge_pm_py.event_log.py_test_event_log()

In [ ]:
# Python-side PyBridgeEventLog construction: Slow!
import pandas as pd
def pm4py_log_to_bridge_log(df: pd.DataFrame):
  log = rust_bridge_pm_py.native.PyBridgeEventLog()
  for trace_id,a in df.groupby(['case:concept:name']):
    trace = rust_bridge_pm_py.native.PyBridgeTrace(str(trace_id))
    for (label,series) in a.iterrows():
      event = rust_bridge_pm_py.native.PyBridgeEvent({k: str(v) for (k,v) in series.to_dict().items()})
      trace.append_event(event)
    log.append_trace(trace)
  return log

In [ ]:
# Sample call: Add artificial start and end activities to every trace
res_log = rust_bridge_pm_py.native.test_bridge_log(log)
assert res_log.traces[0].events[0].attributes.get("concept:name") == "__START__"
assert res_log.traces[0].events[-1].attributes.get("concept:name") == "__END__"

In [ ]:
res['arcs'][0]

In [ ]:
import pm4py
from pm4py.objects.petri_net.obj import PetriNet


def petrinet_to_dict(net: PetriNet) -> dict:
    places = {p.name: {"id": p.name} for p in net.places}
    transitions = {t.name: {"id": t.name, "label": t.label} for t in net.transitions}
    arcs = [
        {
            "from_to": {
                "type": "PlaceTransition"
                if type(arc.source) == PetriNet.Place
                else "TransitionPlace",
                "nodes": [arc.source.name, arc.target.name],
            },
            "weight": arc.weight,
        }
        for arc in net.arcs
    ]
    return {"places": places, "transitions": transitions, "arcs": arcs}

In [ ]:
petrinet_to_dict(dict_to_petrinet(res)) == res

In [ ]:
import pm4py
from pm4py.objects.petri_net.obj import PetriNet


def dict_to_petrinet(net_dict) -> PetriNet:
    places = {p["id"]: PetriNet.Place(p["id"]) for p in net_dict["places"].values()}
    transitions = {
        t["id"]: PetriNet.Transition(t["id"], t["label"])
        for t in net_dict["transitions"].values()
    }

    def get_arc_for(arc_dict):
        if arc_dict["from_to"]["type"] == "PlaceTransition":
            fr = places.get(arc_dict["from_to"]["nodes"][0])
            to = transitions.get(arc_dict["from_to"]["nodes"][1])
        else:
            fr = transitions.get(arc_dict["from_to"]["nodes"][0])
            to = places.get(arc_dict["from_to"]["nodes"][1])
        return PetriNet.Arc(fr, to, arc_dict["weight"])

    arcs = [get_arc_for(arc_dict) for arc_dict in net_dict["arcs"]]
    net = PetriNet(None, places.values(), transitions.values(), arcs)
    return net

In [ ]:
pm4py.view_petri_net(dict_to_petrinet(res))

In [ ]:
import rust_bridge_pm_py
import json
res = json.loads(rust_bridge_pm_py.native.test_petrinet())
res

In [ ]:
# Transform PyBridgeEventLog to dict (only keeping activity names + case id)
traces = []
for trace in log.traces:
  events = []
  for event in trace.events:
    events.append({"concept:name": event.attributes.get("concept:name")})
  traces.append({"case:concept:name": trace.attributes.get("case:concept:name"),"events": events})
